# Import modules

In [24]:
import copy
import matplotlib.pyplot as plt
import os
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms

# Utils

In [2]:
def display_formatted_time(elapsed_time, msg=""):
    minutes, seconds = map(int, divmod(elapsed_time, 60));
    print("Elapsed time - {0}: {1}min {2}s".format(msg, minutes, seconds))

# Define the network

In [27]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)  # 
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

print(Net())

Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)


In [4]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)  # Negative Log Likelihood Loss
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()
            ))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction="sum").item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

        test_loss /= len(test_loader.dataset)
        print("\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
            test_loss, correct, len(test_loader.dataset),
            100. * correct / len(test_loader.dataset)
        ))

# Settings

In [19]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print("Device: ", device)

torch.manual_seed(1)

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}


batch_size = 64
test_batch_size = 1000
epochs = 10
log_interval = 10
lr = 0.01
momentum = 0.5

Device:  cuda


# Load data

In [56]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../../data',
                                train=True,
                                download=True,
                                transform=transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.1307,), (0.3081,))
                                ])
    ),
    batch_size=batch_size,
    shuffle=True,
    **kwargs
)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../../data',
                                train=False,
                                transform=transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.1307,), (0.3081,))
                                ])
    ),
    batch_size=test_batch_size,
    shuffle=True,
    **kwargs
)

# Baseline model Train / Test

In [21]:
since = time.time()

model = Net().to(device)
optimizer = optim.SGD(model.parameters(),
                                         lr=lr,
                                         momentum=momentum
                      )

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

display_formatted_time(time.time() - since)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.373651
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.310517
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.281828
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.315809
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.235439
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.234249
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.226109
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.228646
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.132810
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.113178
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.030113
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.877118
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.894014
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.725610
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.739437
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.533461
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.549235
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.498123
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.502510
Train Epoch: 1 [12160/60000 (20%)]	Loss: 1.326280
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.311428
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.418847
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.365565
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.442371
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.230422
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.196519
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.287909
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.325546
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.304085
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.436313
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.466097
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.233948
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.305664
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.273486
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.238151
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.200522
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.276796
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.246748
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.415465
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.454326


Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.144923
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.225844
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.146612
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.297787
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.253059
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.288946
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.259044
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.262292
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.087290
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.419590
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.124532
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.284295
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.190094
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.175150
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.309617
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.330643
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.201196
Train Epoch: 4 [41600/60000 (69%)]	Loss: 0.353865
Train Epoch: 4 [42240/60000 (70%)]	Loss: 0.208475
Train Epoch: 4 [42880/60000 (71%)]	Loss: 0.427537


Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.352912
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.138169
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.120790
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.198558
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.246087
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.160627
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.193339
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.242042
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.363245
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.409146
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.233149
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.157871
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.129279
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.169030
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.209730
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.138207
Train Epoch: 6 [25600/60000 (43%)]	Loss: 0.264586
Train Epoch: 6 [26240/60000 (44%)]	Loss: 0.265089
Train Epoch: 6 [26880/60000 (45%)]	Loss: 0.366829
Train Epoch: 6 [27520/60000 (46%)]	Loss: 0.074145



Test set: Average loss: 0.0659, Accuracy: 9799/10000 (98%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.280147
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.121714
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.133031
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.164345
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.117662
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.268908
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.103779
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.164005
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.071415
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.184921
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.264299
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.075380
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.384946
Train Epoch: 8 [8320/60000 (14%)]	Loss: 0.078368
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.158650
Train Epoch: 8 [9600/60000 (16%)]	Loss: 0.309467
Train Epoch: 8 [10240/60000 (17%)]	Loss: 0.106293
Train Epoch: 8 [10880/60000 (18%)]	Loss: 0.163516
Train Epoch: 8 [11520/60000 (19%)]	Loss: 0.081925
Train Epoch: 8 [12

Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.142114
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.216210
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.138235
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.126225
Train Epoch: 9 [48640/60000 (81%)]	Loss: 0.182767
Train Epoch: 9 [49280/60000 (82%)]	Loss: 0.118665
Train Epoch: 9 [49920/60000 (83%)]	Loss: 0.151375
Train Epoch: 9 [50560/60000 (84%)]	Loss: 0.175195
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.056166
Train Epoch: 9 [51840/60000 (86%)]	Loss: 0.101019
Train Epoch: 9 [52480/60000 (87%)]	Loss: 0.131079
Train Epoch: 9 [53120/60000 (88%)]	Loss: 0.225312
Train Epoch: 9 [53760/60000 (90%)]	Loss: 0.186148
Train Epoch: 9 [54400/60000 (91%)]	Loss: 0.113478
Train Epoch: 9 [55040/60000 (92%)]	Loss: 0.151708
Train Epoch: 9 [55680/60000 (93%)]	Loss: 0.046165
Train Epoch: 9 [56320/60000 (94%)]	Loss: 0.180704
Train Epoch: 9 [56960/60000 (95%)]	Loss: 0.176970
Train Epoch: 9 [57600/60000 (96%)]	Loss: 0.295399
Train Epoch: 9 [58240/60000 (97%)]	Loss: 0.097744


___

# LR Scheduler

In [22]:
optimizer_ft = optim.SGD(model.parameters(),
                                              lr=lr,
                                              momentum=momentum)
scheduler = lr_scheduler.StepLR(optimizer=optimizer_ft,
                                                         step_size=2.,  # {step} epoch ごとにLRを減少させる
                                                         gamma=0.1)  # step ごとに lr = lr * gamma

In [23]:
since = time.time()

model = Net().to(device)
optimizer = optim.SGD(model.parameters(),
                                         lr=lr,
                                         momentum=momentum)

for epoch in range(1, epochs + 1):
    scheduler.step()
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

display_formatted_time(time.time() - since)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.354365
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.297701
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.291977
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.260900
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.262597
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.283646
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.213664
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.182343
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.219697
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.101493
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.057653
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.111837
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.852859
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.800821
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.659863
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.386412
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.407535
Train Epoch: 1 [10880/60000 (18%)]	Loss: 1.320320
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.579342
Train Epoch: 1 [12160/60000 (20%)]	Loss: 1.304948
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.398749
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.296236
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.699186
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.353746
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.198119
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.289424
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.620188
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.329816
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.451333
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.451260
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.358147
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.465794
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.360739
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.318966
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.406873
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.440268
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.353843
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.259376
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.674595
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.447692


Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.197758
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.198699
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.385369
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.278064
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.264241
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.305909
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.098873
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.193280
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.218317
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.417809
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.450987
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.425331
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.336592
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.311778
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.166080
Train Epoch: 4 [40320/60000 (67%)]	Loss: 0.254041
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.303528
Train Epoch: 4 [41600/60000 (69%)]	Loss: 0.133274
Train Epoch: 4 [42240/60000 (70%)]	Loss: 0.257420
Train Epoch: 4 [42880/60000 (71%)]	Loss: 0.215120


Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.268840
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.425545
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.247635
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.175950
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.226564
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.231114
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.181230
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.278410
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.246110
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.247761
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.237484
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.124863
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.240933
Train Epoch: 6 [23680/60000 (39%)]	Loss: 0.138735
Train Epoch: 6 [24320/60000 (41%)]	Loss: 0.068759
Train Epoch: 6 [24960/60000 (42%)]	Loss: 0.159897
Train Epoch: 6 [25600/60000 (43%)]	Loss: 0.198312
Train Epoch: 6 [26240/60000 (44%)]	Loss: 0.135479
Train Epoch: 6 [26880/60000 (45%)]	Loss: 0.202203
Train Epoch: 6 [27520/60000 (46%)]	Loss: 0.149269



Test set: Average loss: 0.0589, Accuracy: 9820/10000 (98%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 0.135793
Train Epoch: 8 [640/60000 (1%)]	Loss: 0.290392
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.210629
Train Epoch: 8 [1920/60000 (3%)]	Loss: 0.190596
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.063908
Train Epoch: 8 [3200/60000 (5%)]	Loss: 0.180491
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.379197
Train Epoch: 8 [4480/60000 (7%)]	Loss: 0.214707
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.385705
Train Epoch: 8 [5760/60000 (10%)]	Loss: 0.146705
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.081479
Train Epoch: 8 [7040/60000 (12%)]	Loss: 0.193262
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.230478
Train Epoch: 8 [8320/60000 (14%)]	Loss: 0.060484
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.150403
Train Epoch: 8 [9600/60000 (16%)]	Loss: 0.255851
Train Epoch: 8 [10240/60000 (17%)]	Loss: 0.211595
Train Epoch: 8 [10880/60000 (18%)]	Loss: 0.234529
Train Epoch: 8 [11520/60000 (19%)]	Loss: 0.158107
Train Epoch: 8 [12

Train Epoch: 9 [45440/60000 (76%)]	Loss: 0.351112
Train Epoch: 9 [46080/60000 (77%)]	Loss: 0.201940
Train Epoch: 9 [46720/60000 (78%)]	Loss: 0.228879
Train Epoch: 9 [47360/60000 (79%)]	Loss: 0.357308
Train Epoch: 9 [48000/60000 (80%)]	Loss: 0.251754
Train Epoch: 9 [48640/60000 (81%)]	Loss: 0.125475
Train Epoch: 9 [49280/60000 (82%)]	Loss: 0.185639
Train Epoch: 9 [49920/60000 (83%)]	Loss: 0.172161
Train Epoch: 9 [50560/60000 (84%)]	Loss: 0.221257
Train Epoch: 9 [51200/60000 (85%)]	Loss: 0.122792
Train Epoch: 9 [51840/60000 (86%)]	Loss: 0.097859
Train Epoch: 9 [52480/60000 (87%)]	Loss: 0.121299
Train Epoch: 9 [53120/60000 (88%)]	Loss: 0.097112
Train Epoch: 9 [53760/60000 (90%)]	Loss: 0.160477
Train Epoch: 9 [54400/60000 (91%)]	Loss: 0.107431
Train Epoch: 9 [55040/60000 (92%)]	Loss: 0.126314
Train Epoch: 9 [55680/60000 (93%)]	Loss: 0.149048
Train Epoch: 9 [56320/60000 (94%)]	Loss: 0.127637
Train Epoch: 9 [56960/60000 (95%)]	Loss: 0.139482
Train Epoch: 9 [57600/60000 (96%)]	Loss: 0.164835


___

# Transfer learning

実行せず書き方のみ参照すること

In [70]:
batch_size = 64
test_batch_size = 10
epochs = 10
log_interval = 10
lr = 0.01
momentum = 0.9

In [72]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../../data',
                                train=True,
                                download=True,
                                transform=transforms.Compose([
                                    transforms.RandomResizedCrop(224),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.1307,), (0.3081,))
                                ])
    ),
    batch_size=batch_size,
    shuffle=True,
    **kwargs
)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../../data',
                                train=False,
                                transform=transforms.Compose([
                                    transforms.Resize(256),
                                    transforms.CenterCrop(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.1307,), (0.3081,))
                                ])
    ),
    batch_size=test_batch_size,
    shuffle=True,
    **kwargs
)

## Finetuning

In [73]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features

# Input, Output をMNISTに合わせる
model_ft.conv1 = nn.Conv2d(in_channels=1,
                                                    out_channels=64,
                                                    kernel_size=7,
                                                    stride=2,
                                                    padding=3,
                                                    bias=False)
model_ft.fc = nn.Linear(num_ftrs, 10)

model_ft = model_ft.to(device)
print(model_ft)

# criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(model_ft.parameters(),
                                              lr=lr,
                                              momentum=momentum)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft,
                                                                      step_size=2,
                                                                      gamma=0.1)

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

## Train and evaluate

In [ ]:
since = time.time()

for epoch in range(1, epochs + 1):
    exp_lr_scheduler.step()
    train(model_ft, device, train_loader, optimizer_ft, epoch)
    test(model_ft, device, test_loader)

display_formatted_time(time.time() - since)